In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [2]:
df1 = pd.read_csv('~/data/ctown/dataset03.csv')
df2 = pd.read_csv('~/data/ctown/dataset04.csv')

train_normal = pd.concat((df1,df2[df2['ATT_FLAG']==0]),axis=0,ignore_index=True)
train_attack = df2[df2['ATT_FLAG']==1]

In [3]:
sensors = [col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG']]

In [4]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = train_attack[sensors].reset_index().drop(columns=['index'])

In [5]:
hankel = Hankel()
lag = 60
stride = 0.5

In [6]:
df_test = pd.read_csv('~/data/ctown/test_dataset.csv')

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [7]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
Y_test = test_combined.loc[:,'ATT_FLAG']

In [8]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [9]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,1,False)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [10]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)

In [11]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.788235294117647
Precision  0.7692307692307693
Recall  0.8695652173913043
F1-score  0.8163265306122449


In [12]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5028727770177839 Median : 0.5058823529411764 Min : 0.4470588235294118 Max : 0.6235294117647059
precision - Mean: 0.6798311184939092 Median : 1.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.08998988877654196 Median : 0.08695652173913043 Min : 0.0 Max : 0.32608695652173914
f1 -        Mean: 0.15410990256134335 Median : 0.16 Min : 0.0 Max : 0.48387096774193544


# Multiple clusters + No threshold tuning (training attack mixed in test data)

In [13]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,tune=False)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [14]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)

In [15]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.5411764705882353
Precision  0.5411764705882353
Recall  1.0
F1-score  0.7022900763358779


In [16]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5105335157318742 Median : 0.5058823529411764 Min : 0.43529411764705883 Max : 0.6470588235294118
precision - Mean: 0.48810869247973887 Median : 0.5918367346938775 Min : 0.0 Max : 1.0
recall -    Mean: 0.22093023255813954 Median : 0.2391304347826087 Min : 0.0 Max : 0.6304347826086957
f1 -        Mean: 0.2842612930740881 Median : 0.36065573770491804 Min : 0.0 Max : 0.6105263157894736


# Multiple clusters + Threshold tuning (No concat of training and test data)

In [19]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'ATT_FLAG']

In [20]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [21]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [22]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)

In [23]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969


In [24]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5656634746922024 Median : 0.5735294117647058 Min : 0.4852941176470588 Max : 0.6470588235294118
precision - Mean: 0.34877877076101665 Median : 0.43478260869565216 Min : 0.0 Max : 0.75
recall -    Mean: 0.19165998396150766 Median : 0.1724137931034483 Min : 0.0 Max : 0.5862068965517241
f1 -        Mean: 0.22309983286002688 Median : 0.2702702702702703 Min : 0.0 Max : 0.5555555555555556
